In [25]:
%%sh
# run shell scripts to have an easy interface to experiment and get started with DPNE
# This script runs a series of tokenization, ngram extraction, and DP N-gram extraction using the parameters specified as arguments
# first argument = location of source file to extract DPNE from
SOURCE_DATASET="./normalized_rephrased_wildchat_data.json"

# second argument = output folder high level
OUTPUT_FOLDER="output_norm_wildchat10k_eps4"

# third argument = file extension - json or anything else.
FILE_EXTENSION="json"

# fourth argument = epsilon for DP
DP_EPSILON="4.0"

# fifth argument = the highest N in n-grams to DP-extract
NGRAM_SIZE_LIMIT="10"

#spark-submit dpne/tokenize_text.py -f json --ngrams $NGRAM_SIZE_LIMIT --max_num_tokens 400 --allow_multiple_ngrams 1 -i $SOURCE_DATASET -o ./$OUTPUT_FOLDER/tokenize_text -t $FILE_EXTENSION

#spark-submit dpne/split_ngrams.py --ngram_size $NGRAM_SIZE_LIMIT -i ./$OUTPUT_FOLDER/tokenize_text -o ./$OUTPUT_FOLDER/split_ngrams -f $FILE_EXTENSION -t $FILE_EXTENSION

# modified the hyperparams a bit
spark-submit dpne/extract_dpne.py --dp_epsilon $DP_EPSILON --dp_eta 0.1 --dp_delta 0.5 --contribution_limit 10 --persist_flags 11 --log_flags 00 --top_k 1 --delta_user_count 0 --ngram_size $NGRAM_SIZE_LIMIT --filter_one_side 0 --budget_distribute 10.0 --estimate_sample_size 0.8 -i ./$OUTPUT_FOLDER/split_ngrams -o ./$OUTPUT_FOLDER/dpne_sample -f $FILE_EXTENSION -t $FILE_EXTENSION
